In [ ]:
# !pip -qq install focal-loss

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import random
import gc
from tqdm import tqdm
import seaborn as sns
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Model
warnings.filterwarnings("ignore")
# from focal_loss import SparseCategoricalFocalLoss
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dropout, BatchNormalization, Dense, Conv2D, Flatten

In [ ]:
df = pd.read_csv('../input/cmsnewsamples/new-smaples.csv').drop(columns = 'Unnamed: 0')

In [ ]:
df = pd.read_csv('../input/cmsnewsamples/new-smaples.csv').drop(columns = 'Unnamed: 0')
df = df.drop(columns = [i for i in df.columns if '_1' in i])
df['non_hits'] = df[[i for i in df.columns if 'mask' in i]].sum(axis=1)
df = df[df['non_hits']==0].reset_index(drop=True)

df['1/pT'] = df['q/pt'].abs()
def label(a):
    if a<=10:
        return 0
    if a>10 and a<=30:
        return 1
    if a>30 and a<=100:
        return 2
    if a>100:
        return 3

df['pT'] = 1/df['1/pT']
    
df['pT_classes'] = df['pT'].apply(label)

features = ['emtf_phi_'+str(i) for i in [0,2,3,4]] + ['emtf_theta_'+str(i) for i in [0,2,3,4]] + ['fr_'+str(i) for i in [0,2,3,4]] + ['old_emtf_phi_'+str(i) for i in [0,2,3,4]]
labels_1 = ['pT']
labels_2 = ['pT_classes']
labels_3 = ['vx']

scaler_1 = StandardScaler()
df[features] = scaler_1.fit_transform(df[features])

scaler_3 = MinMaxScaler()
df[labels_3] = scaler_3.fit_transform(df[labels_3])

In [ ]:
gc.collect()

In [ ]:
shuffled_list = list(range(len(df)))
random.Random(242).shuffle(shuffled_list)
shuffled_list = np.array_split(np.array(shuffled_list), 10)

In [ ]:
OOF_preds = pd.DataFrame()
OOF_preds['row'] = []
OOF_preds['true_value'] = []
OOF_preds['preds'] = []
# OOF_preds['pT_classes'] = []
# OOF_preds['0-10'] = []
# OOF_preds['10-30'] = []
# OOF_preds['30-100'] = []
# OOF_preds['100-inf'] = []

In [ ]:
def loss(y_true,y_pred):
    y_t = K.cast(y_true<80,K.dtype(y_true))*y_true + K.cast(y_true>=80,K.dtype(y_true))*K.cast(y_true<250,K.dtype(y_true))*y_true*2.4 + K.cast(y_true>=160,K.dtype(y_true))*10 
    return K.mean(y_t*K.pow((y_pred-y_true)/y_true,2))/250

In [ ]:
def swish(x):
    return x * K.sigmoid(x)

In [ ]:
def FCNN(X_train, Y1_train, Y2_train, Y3_train):
    
    I = Input(shape=(4,4,1))
    x = Conv2D(64, kernel_size=(2, 2),activation=swish)(I)
    x = Flatten()(x)
    x = Dense(128, activation=swish)(x)
    x = Dropout(0.1)(x)
    x1 = Dense(32, activation=swish)(x)
    x1 = Dropout(0.1)(x1)
    x1 = Dense(16, activation=swish)(x1)
    x1 = Dense(1, activation='linear')(x1)
    model = Model(inputs=I, outputs=x1)

    batch_size=128
    path = "model.h5"

    checkpoint = ModelCheckpoint(path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_loss',patience=10,verbose=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1,verbose=True)

#     model.compile(optimizer = 'adam', loss=['mse', SparseCategoricalFocalLoss(gamma=2),'mse'], loss_weights = [70, 1, 70])
    model.compile(optimizer = 'adam', loss=loss)
    model.summary()

    history = model.fit(x=X_train, y=Y1_train, batch_size=batch_size, epochs=100, verbose=0, validation_split=0.11, callbacks=[checkpoint,early_stop,reduce_lr])

    model.load_weights(path)
    
    return model

In [ ]:
for i in range(10):
    X_train = df[features].iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]
    Y1_train = df[labels_1].iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]
    Y2_train = df[labels_2].astype('float32').iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]
    Y3_train = df[labels_3].iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]

    X_test = df[features].iloc[shuffled_list[i]]
    Y1_test = df[labels_1].iloc[shuffled_list[i]]
    Y2_test = df[labels_2].astype('float32').iloc[shuffled_list[i]]
    Y3_test = df[labels_3].iloc[shuffled_list[i]]
    
    X_train = X_train.to_numpy().reshape((-1,4,4,1))
    X_test = X_test.to_numpy().reshape((-1,4,4,1))
    
    model = FCNN(X_train, Y1_train, Y2_train, Y3_train)
    
    P = model.predict(X_test)
    
    test_preds_1 = P.reshape((len(X_test)))
#     test_preds_1 = P[0].reshape((len(X_test)))
#     test_preds_2 = P[1]
    
    OOF_preds_ = pd.DataFrame()
    OOF_preds_['row'] = shuffled_list[i]
    OOF_preds_['true_value'] = Y1_test['pT'].to_list()
    OOF_preds_['preds'] = test_preds_1
#     OOF_preds_['pT_classes'] = Y2_test['pT_classes'].values
#     OOF_preds_['0-10'] = test_preds_2[:,0].reshape((len(X_test)))
#     OOF_preds_['10-30'] = test_preds_2[:,1].reshape((len(X_test)))
#     OOF_preds_['30-100'] = test_preds_2[:,2].reshape((len(X_test)))
#     OOF_preds_['100-inf'] = test_preds_2[:,3].reshape((len(X_test)))
    
    OOF_preds = pd.concat([OOF_preds,OOF_preds_],axis = 0).reset_index(drop = True)
    
    X_train, Y1_train, Y2_train, Y3_train, X_test, Y1_test, Y2_test, Y3_test, model, P, test_preds_1, test_preds_2, OOF_preds_ = [0]*13
    gc.collect()

In [ ]:
OOF_preds = OOF_preds.sort_values(by = 'row').reset_index(drop = True)
OOF_preds.to_csv('OOF_preds.csv')

In [ ]:
df = pd.read_csv('OOF_preds.csv').drop(columns = ['Unnamed: 0'])
df = df.sort_values(by = 'row').reset_index(drop = True)
df['True_pT'] = df['true_value']
df['Predicted_pT'] = df['preds']

In [ ]:
MAE1 = []
dx = 0.5
for i in tqdm(range(int(2/dx),int(150/dx))):
    P = df[(df['True_pT']>=(i-1)*dx)&(df['True_pT']<=(i+1)*dx)]
    try:
        p = mae(P['True_pT'],P['Predicted_pT'])
    except:
        p=0
    MAE1.append(p)
MAE1 = MAE1[:146]
plt.plot([i*dx for i in range(int(75/dx))],[0]*int(int(75/dx)-len(MAE1))+MAE1,label = 'FCNN')

In [ ]:
sns.distplot(df[(df['preds']>0) & (df['preds']<=0.5)]['preds'])
plt.ylabel('pdf')
plt.xlabel('1/pT')
plt.title('Distribution of Predicted 1/pT')
plt.show()

In [ ]:
def pT_classes(a):
    if a<=10:
        return 0
    if a>10 and a<=30:
        return 1
    if a>30 and a<=100:
        return 2
    if a>100:
        return 3

print(classification_report(df['True_pT'].apply(pT_classes), df['Predicted_pT'].apply(pT_classes)))

In [ ]:
# print(classification_report(df['True_pT'].apply(pT_classes), df.iloc[:,4:8].to_numpy().argmax(axis = 1)))

In [ ]:
confusion_matrix(df['True_pT'].apply(pT_classes), df['Predicted_pT'].apply(pT_classes))

In [ ]:
# confusion_matrix(df['True_pT'].apply(pT_classes), df.iloc[:,4:8].to_numpy().argmax(axis = 1))